In [ ]:
from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
def convert_to_png(npz_files, rows, columns):
    """
    Given a list of npz filenames, this function reads each npz file, extracts the 12-band geospatial images,
    and saves them as a sequence of PNG images.
    
    Parameters:
    npz_files: list of str, names of the npz files to read
    rows: int, number of rows for sub-images
    columns: int, number of columns for sub-images
    """
    if len(npz_files) != rows * columns:
        print("The number of npz files must be equal to rows x columns.")
        return

    # Load all npz files into memory first
    loaded_npzs = [np.load(npz_file).get("chips") for npz_file in npz_files]
    
    # Determine the maximum time steps across all loaded npz files
    max_time_steps = max(chips.shape[0] for chips in loaded_npzs)

    for time_step in tqdm(range(max_time_steps)):
        # Create a blank (black) image to paste the sub-images onto
        combined_image = Image.new('RGB', (columns * 512, rows * 512), 'black')

        for idx, chips in enumerate(loaded_npzs):
            # Check if the current time_step is within bounds for this chip
            if time_step >= chips.shape[0]:
                continue

            try:
                # Extract the image for the current time step
                image_data = chips[time_step, [3, 2, 1], :, :]

                # Clamp values between 0 and 2500
                image_data = np.clip(image_data, 0, 2500).astype(np.float32)

                # Convert to RGB format suitable for PNG
                image_data = image_data / 2500
                image_data = (image_data * 0xff).astype(np.uint8)
                
                # Transpose the data to meet the RGB channel last format
                image_data = np.transpose(image_data, (1, 2, 0))

                # Create a PIL image from the numpy array
                pil_image = Image.fromarray(image_data, 'RGB')

                # Determine where to paste this image on the combined image
                x_offset = (idx % columns) * 512
                y_offset = (idx // columns) * 512

                # Paste the image
                combined_image.paste(pil_image, (x_offset, y_offset))

            except Exception as e:
                print(f"An error occurred while processing the image: {e}")

        # Save the combined image as a PNG file
        combined_image.save(f"{time_step:05}.png")


In [ ]:
!ls

In [ ]:
!find /datasets/foss4g-data/meadow | grep chip.npz$ | sort -R | head -1

In [ ]:
!find /datasets/foss4g-data/industrial | grep chip.npz$ | sort -R | head -1

In [ ]:
!find /datasets/foss4g-data/farmland | grep chip.npz$ | sort -R | head -1

In [ ]:
!find /datasets/foss4g-data/vineyard | grep chip.npz$ | sort -R | head -1

In [ ]:
!find /datasets/foss4g-data/grass | grep chip.npz$ | sort -R | head -1

In [ ]:
!find /datasets/foss4g-data/orchard | grep chip.npz$ | sort -R | head -1

In [ ]:
npz_list = [
    "/datasets/foss4g-data/meadow/33TWM/1202303122202/chip/359083c3f11c40be9f5d1244bc212298.chip.npz",
    "/datasets/foss4g-data/industrial/38VPH/1210222331021/chip/c37cae4398ed4cecba4d754a37ab275d.chip.npz",
    "/datasets/foss4g-data/farmland/38KQV/3012000311102/chip/d7abba52bf814680b16dbe91cced6be1.chip.npz",
    "/datasets/foss4g-data/vineyard/30SVJ/0331110313000/chip/4058cd26558a486c9b541597e467d1dc.chip.npz",
    "/datasets/foss4g-data/grass/37UCB/1203101001123/chip/6fe1de63e8734d2daa1952c8caa37e9d.chip.npz",
    "/datasets/foss4g-data/orchard/31TCG/1202222203110/chip/5ed0bed63a5b411abae869c2b456a77b.chip.npz",
]

In [ ]:
convert_to_png(npz_list, 2, 3)

In [ ]:
!ls

In [ ]:
!rm *.png

ffmpeg -framerate 1 -i %05d.png -vf "loop=0" -y output.gif

In [ ]:
!ls /datasets/sentinel2-timeseries/Giza/chip/

In [ ]:
convert_to_png(["/datasets/sentinel2-timeseries/Charlston_SC/chip/87239307520f44c2be8f5b62ea007adf.chip.npz"], 1, 1)

In [ ]:
!rm *.png